# Libraries

In [19]:
#Import Libraries
import os

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import random
import numpy as np
import torch
from torch.autograd import Variable
import requests

import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import math
from tqdm import tqdm_notebook, trange

import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

### Cuda debugging

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Dataset Preprocessing

In [3]:
class Dataset(Dataset):
    def __init__(self, X, Y):
        self.x = X
        self.y = Y
        
    # def __getitem__(self, index):
    #     return self.x[index], self.y[index]
    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.x.shape[0]


# Parse fasta file and create training data

In [4]:
# Load Fasta with biopython parser
from Bio import SeqIO
import pandas as pd
# Create list for pos training data
my_seqlist = []
for seq_record in SeqIO.parse(open('/home/jovyan/data1/jjaureguy/nn_fasta/GSE168881/SRR13961069.fa.out'),'fasta'):
    my_seqlist.append(seq_record)
my_seqlist[0].seq

print(my_seqlist[0])
print(len(my_seqlist))



ID: chr1:925238-926238
Name: chr1:925238-926238
Description: chr1:925238-926238
Number of features: 0
Seq('acagcagtatcctacctaagaagacttttgcccaaggtctttccaaacccaaga...cca')
7291


In [5]:
bases = 'ACTG'
base_map = dict(zip(bases, range(len(bases))))
n_classes = len(base_map)
base_map, n_classes

({'A': 0, 'C': 1, 'T': 2, 'G': 3}, 4)

In [6]:
np.random.seed(42)

In [7]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [8]:
pos_seqlist = []
neg_seqlist = []

skipped = 0
for i in my_seqlist:
    fasta = i.seq
    as_text = fasta.upper()
    if 'N' in as_text:
        skipped += 1
    else:
        seq = np.vectorize(base_map.get)(np.array(list(as_text)))
        pos_seqlist.append(seq)
        neg_seqlist.append(np.random.choice(seq, size=len(seq), replace=False))
    
seqlist = np.stack(pos_seqlist + neg_seqlist)
#seqlist = list(filter(None, seqlist))  # This removes empty sequences.
n_samples = len(seqlist)
n_samples, skipped


(14566, 8)

In [9]:
training_data = one_hot(seqlist, n_classes).reshape(n_samples, -1, n_classes)
training_data.shape
#training_data.ndim


(14566, 1000, 4)

In [10]:
x_log = training_data

# Create labels(response variable Y)

In [11]:
response_var_pos = [1]*(n_samples//2)
response_var_neg = [0]*(n_samples//2)

total_response_var = response_var_pos + response_var_neg
print(len(total_response_var))
print(total_response_var[7282:7285])
total_response_var = np.array(total_response_var)
total_response_var.shape
total_response_var.ndim

14566
[1, 0, 0]


1

In [12]:

types = len(total_response_var)

# Moving the model to GPU

In [13]:
torch.cuda.is_available()
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()
device

device(type='cuda')

# Splitting data set(training and test split)

In [14]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

#print(x_log.shape)
#print(total_response_var.shape)

X_train, X_test, y_train, y_test = train_test_split(x_log, total_response_var, train_size=0.8, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11652, 1000, 4)
(2914, 1000, 4)
(11652,)
(2914,)


# Split training set into validation

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.9, random_state=42)

In [16]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(10486, 1000, 4)
(1166, 1000, 4)
(10486,)
(1166,)


# Converting dataset from np array to tensor 

In [ ]:
X_train = torch.from_numpy(X_train.astype(np.float32)).to(device)
X_train.requires_grad = True
X_test = torch.from_numpy(X_test.astype(np.float32)).to(device)
X_test.requires_grad = True
y_train = torch.from_numpy(y_train.astype(np.float32)).to(device)
y_train.requires_grad = True
y_test = torch.from_numpy(y_test.astype(np.float32)).to(device)
y_test.requires_grad = True
X_valid = torch.from_numpy(X_valid.astype(np.float32)).to(device)
X_valid.requires_grad = True
y_valid = torch.from_numpy(y_valid.astype(np.float32)).to(device)
y_valid.requires_grad = True

In [ ]:
y_valid.type()



In [ ]:
y_test

In [ ]:
y_test

# Apply DataSet function

In [17]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)
y_valid = y_valid.reshape(y_valid.shape[0],1)

In [18]:
# 72%-20%-8% split
y_test.shape

(2914, 1)

In [ ]:
train_dataset = Dataset(X_train, y_train)
valid_dataset = Dataset(X_valid, y_valid)
test_dataset = Dataset(X_test, y_test)

In [ ]:
len(train_dataset)

In [ ]:
len(train_dataset.x)



In [ ]:
(train_dataset.y[0])

In [ ]:
print(len(train_dataset))
print(train_dataset.y[0])
#print(train_dataset[1])



In [ ]:
train_dataset.x.shape[1:]

In [ ]:
train_dataset.x

In [ ]:
train_dataset.y

# Models

## Logistic Regression model

### Log regression var


In [ ]:
training_data.reshape(n_samples, -1).shape
x_log = training_data.reshape(n_samples, -1)
x_log.shape

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

print(x_log.shape)
print(total_response_var.shape)

X_train, X_test, y_train, y_test = train_test_split(x_log, total_response_var, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import torch
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [ ]:
#Initialize model
epochs = 10000
input_dim = 4000  # Two inputs x1 and x2 
output_dim = 1 # Single binary output 
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [ ]:
X_train = torch.tensor(X_train,dtype=torch.float32, requires_grad=True)
X_test = torch.tensor(X_test,dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y_train,dtype=torch.float32, requires_grad=True)
y_test = torch.tensor(y_test,dtype=torch.float32,requires_grad=True)

In [ ]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [ ]:
X_train = torch.tensor(X_train,requires_grad=True)
X_test = torch.tensor(X_test,requires_grad=True)
y_train = torch.tensor(y_train,requires_grad=True)
y_test = torch.tensor(y_test,requires_grad=True)

In [ ]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [ ]:
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    #backward pass
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    optimizer.zero_grad()
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')

## Fully connected Neural net

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = torch.sigmoid(out)
        return out

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)


In [ ]:
#model = NeuralNet(input_size, hidden_size, num_classes).to(device)
# Loss and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
#Loop
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            
    
    
    #forward pass and loss
    y_predicted = model(X_train)
    # print(y_predicted)
    loss = criterion(y_predicted, y_train)
    # print(loss)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')
        y_test_preds = model(X_test)
        test_loss = criterion(y_test_preds, y_test)
        print('test loss: ' + str(test_loss))

        
        

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')
    # test_loss = criterion(y_predicted, y_test)

In [ ]:
print(input_size)

## CNN

In [ ]:
import torch.nn.functional as F

In [ ]:
#Variables
input_size = 4
num_classes = 1
epochs = 1000
# batch_size = 100
learning_rate = 0.0001


In [ ]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=16,out_channels=32,kernel_size=3, dilation =1) 
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(7936, 288) 
        self.fc2 = nn.Linear(288, 144) 
        self.fc3 = nn.Linear(144, num_classes) 
       
    def forward(self, x):
        bs, seq_len, num_channels = x.shape
        x = x.view(bs, num_channels, seq_len)
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(bs, -1)
        #print('lnear shape:', out.shape[1])
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        #out = F.relu(self.fc3(out))
        out = self.fc3(out)
        out = torch.sigmoid(out)
        return out


In [ ]:
model = ConvNet(input_size, num_classes).to(device)
#model = Simple(1000, 4, 1).to(device)
#criterion(y_train[0:8], yp)
#yp.squeeze()

In [ ]:
import pandas as pd
s = pd.Series(yp.squeeze().detach().cpu().numpy())
s.hist()

In [ ]:
#model = NeuralNet(input_size, hidden_size, num_classes).to(device)
# Loss and optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
!nvidia-smi

In [ ]:
#Loop
for epoch in range(epochs):#
    
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
            

    #forward pass and loss
    y_predicted = model(X_train)
   
    # print(y_predicted)
    loss = criterion(y_predicted, y_train)
    # print(loss)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #update weights
    optimizer.step()
    #zero out the gradients since backward function always adds up all gradients(empty before next iteration)
    if (epoch+1)%10 ==0:
        print(f'epoch: {epoch+1},loss = {loss.item():.4f}')
        y_test_preds = model(X_test)
        test_loss = criterion(y_test_preds, y_test)
        print('test loss: ' + str(test_loss))

        
        

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    y_train_predicted = model(X_train).round()
    acc = y_train_predicted.eq(y_train).sum() / float(y_train.shape[0])
    print(f'training accuracy = {acc: .4f}')
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc: .4f}')
    # test_loss = criterion(y_predicted, y_test)

In [ ]:
model = ConvNet(input_size, num_classes).to(device)


In [ ]:
opts = {
    'lr': 1e-4,
    'epochs': 1000,
    'batch_size': 100,
    'loss_fxn': 'b',
    'opt': 'Adam'
}

In [ ]:
test_loss, train_loss = [], []
CNNTrainer = TrainHelper(model = model,
                      train_set = train_dataset,
                      test_set = valid_dataset, opts = opts)

In [ ]:
CNNTrainer.train()


# Training and Validation

## Data Loaders

In [ ]:
# batch_size=opts['batch_size']
train_loader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=False, num_workers=2)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False, num_workers=2)

first_data = train_loader.dataset[0]
features,labels = first_data
print(features,labels)


# print(len(train_loader))
# print(train_loader.dataset.x)
# print(train_loader.dataset.y)

 

In [ ]:
first_data = test_loader.dataset[0]
features,labels = first_data
print(features,labels)

In [ ]:
dataiter = iter(train_loader)
data = dataiter.next()
features,labels = data
print(features,labels)

In [ ]:
# Training loop

In [ ]:
#Variables
input_size = 4
num_classes = 1
num_epochs = 5
batch_size = 4
learning_rate = 0.001


In [ ]:
model = ConvNet(input_size, num_classes).to(device = device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
total_samples = len(train_dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)
                    

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs,labels) in enumerate(train_loader):
        # forward pass, backward pass, update
        inputs = inputs.to(device=device, dtype=torch.float)
        labels = labels.to(device=device, dtype=torch.float)
        
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        
        #Backward and optimize
        
        optimizer.zero_grad()
        loss.backward()
        #update weights
        optimizer.step()
        if (i+1) % 2000 ==0:
            print(f'epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("Finish Training")

In [ ]:
# print('Finished Training')
# PATH = './cnn.pth'
# torch.save(model.state_dict(), PATH)

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device = device, dtype=torch.float)
        labels = labels.to(device = device,  dtype=torch.float)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        print(len(labels))
        
        for i in range(batch_size):
            print(i)
            label = labels[i]
            print(len(label))
            pred = predicted[i]
            print(pred)
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

# Sherry's Training Function

In [ ]:
save_model_time = '0'
mkpath = 'model/model%s'% save_model_time
os.makedirs(mkpath)

In [ ]:
from focal_loss.focal_loss import FocalLoss


In [ ]:
class TrainHelper():
    '''
    Helper class that makes it a bit easier and cleaner to define the training routine
    
    '''

    def __init__(self,model,train_set,test_set,opts):
        self.model = model  # neural net
        # device agnostic code snippet
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        
        self.epochs = opts['epochs']
        if opts['opt'] == 'Adam':
            self.optimizer = torch.optim.Adam(model.parameters(), opts['lr']) # optimizer method for gradient descent
        else:
            self.optimizer = torch.optim.SGD(model.parameters(), opts['lr'])
        if opts['loss_fxn'] == 'c':
            self.criterion = torch.nn.CrossEntropyLoss()                      # loss function
        elif opts['loss_fxn'] == 'b':
            self.criterion = torch.nn.BCELoss()                    # loss function used in papers
        elif opts['loss_fxn'] == 'f':
            self.criterion = FocalLoss(alpha=0.25, gamma=2)

        self.train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
        self.valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                      batch_size=opts['batch_size'],
                                                      shuffle=True)
    def train(self):
        self.model.train() # put model in training mode
        for epoch in range(self.epochs):
            self.tr_loss = []
            #105 = 10500/10(# batch size) 
            #print('what is this self.train loader ',len(self.train_loader))
            for i, (data,labels) in tqdm_notebook(enumerate(self.train_loader),total = len(self.train_loader)):
                label_list = []
                
                for i in range(len(labels)):
                    #print('labels at i', labels[i])
                    label_list.append(labels[i].to(self.device))
                data = data.to(self.device)
                self.optimizer.zero_grad()  
                outputs = self.model(data)

                b_list = []
                for i in range(len(label_list)):
                    #print('label list at i', label_list[i])
                    b_list.append(label_list[i])
                    #print('b list', b_list)
                # if opts['loss_fxn'] != 'c':
                #     for i in range(len(label_list)):
                #         b_list[i] = torch.from_numpy(one_hot(labels[i])).to(self.device)

                loss = 0  # define loss
                #print('Test 1', b_list[1])
                #print('Test 2', len(b_list))
                for i in range(len(outputs)):
                    #print(outputs[i], b_list[i])
                    loss += self.criterion(outputs[i], b_list[i])
                    #print('loss',loss)
   
                loss.backward()           
                self.optimizer.step()                  
                self.tr_loss.append(loss.item())       
            if (epoch+1) % 5 == 0 or epoch == 0: # save the model every _ epoch
                torch.save(self.model, 'model/model{save_model_time}/net_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
                torch.save(self.model.state_dict(), 'model/model{save_model_time}/net_params_{epoch}.pkl'.format(save_model_time=save_model_time,epoch=int((epoch+1)/5)))
          
            self.test(epoch) # run through the validation set
    def test(self,epoch):
        self.model.eval()    # puts model in eval mode
        self.test_loss = []
        self.test_accuracy_L = [[] for _ in range(types)]

        for i, (data, labels) in enumerate(self.valid_loader):
            label_list = []
            for i in range(len(labels)):
                label_list.append(labels[i].to(self.device))
            data = data.to(self.device)
            # pass data through network
            # turn off gradient calculation to speed up calcs and reduce memory
            with torch.no_grad():
                outputs = self.model(data)

            # make our predictions and update our loss info
            pred_list = []
            print('length of outputs',len(outputs))
            for i in range(len(outputs)):
                print('outputs index i ',outputs[i])
                _, predicted = torch.max(outputs[i].data, 1)
                pred_list.append(predicted)

            b_list = []
            for i in range(len(label_list)):
                b_list.append(label_list[i])
            if opts['loss_fxn'] != 'c':
                for i in range(len(label_list)):
                    b_list[i] = torch.from_numpy(one_hot(labels[i])).to(self.device)

            loss = 0  # define loss
            for i in range(len(outputs)):
                loss += self.criterion(outputs[i], b_list[i])

            self.test_loss.append(loss.item())

            for i in range(len(pred_list)):
                self.test_accuracy_L[i].append((pred_list[i] == label_list[i]).sum().item() / pred_list[i].size(0))
      
        test_loss.append(np.mean(self.test_loss))
        train_loss.append(np.mean(self.tr_loss))
        av = [np.mean(self.test_accuracy_L[i]) for i in range(types)]
        bestmodel(self.model,save_model_time,np.mean(self.test_loss)) # find best model
        print('epoch: {}, train loss: {}, test loss: {}, test accuracy: {}'.format( 
            epoch+1, np.mean(self.tr_loss), np.mean(self.test_loss), av))